In [ ]:
import pandas
from bs4 import BeautifulSoup
import re
import nltk
#only do next line once
#nltk.download() #download everythin except panlex_lite
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn import cross_validation as cv
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn import grid_search

data = pandas.read_csv("train.tsv",delimiter="\t")
all_data = data["Phrase"][0:20000]
edited_data = []

for x in range(20000):
    rev_soup = BeautifulSoup(data["Phrase"][x])

    letters_only = re.sub("[^a-zA-Z]"," ",rev_soup.get_text())
    
    lower_case = letters_only.lower()
    words = lower_case.split()

    non_stopwords = []
    sw = stopwords.words("english")

    for w in words:
        if w not in sw:
            non_stopwords.append(w)

    words = non_stopwords

    clean_text = " ".join(words)
    edited_data.append(clean_text)

# print(edited_data)
#Bag of Words with 5000 most common words
vectorizer = CountVectorizer(analyzer='word', max_features = 5000)

vectorizer.fit(edited_data)
word_columns = vectorizer.transform(edited_data)

#convert to numpy array so we can feed it
#into learning algorithm
word_columns = word_columns.toarray()
#print(vectorizer.get_feature_names())
#print(word_columns)

(train_data, test_data, train_target, test_target) = cv.train_test_split(word_columns, data["Sentiment"][0:20000],test_size = 0.2)
mnb = MultinomialNB()
mnb.fit(train_data,train_target)
preds = mnb.predict(test_data)
print ("MNB: ",accuracy_score(preds,test_target))

# mlp_classifier = MLPClassifier()
# mlp_classifier.fit(train_data,train_target)
# predictions = mlp_classifier.predict(test_data)
# print("MLPC Accuracy: ",accuracy_score(predictions,test_target))

sv_classifier = svm.SVC(kernel='linear',C=.01)
sv_classifier.fit(train_data,train_target)
predictions = sv_classifier.predict(test_data)
print("Linear Accuracy: ",accuracy_score(predictions,test_target))

sv_classifier = svm.SVC(kernel='poly', C=.01, gamma= 10)
sv_classifier.fit(train_data,train_target)
predictions = sv_classifier.predict(test_data)
print("Poly Accuracy: ",accuracy_score(predictions,test_target))

sv_classifier = svm.SVC(kernel='rbf', C=.01, gamma= 10)
sv_classifier.fit(train_data,train_target)
predictions = sv_classifier.predict(test_data)
print("RBF Accuracy: ",accuracy_score(predictions,test_target))
